In [1]:
from icalendar import Calendar, Event
from urllib.request import urlopen 
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
from sqlalchemy import create_engine, Column, String, Integer, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [16]:
#init connection to sqlite 
engine = create_engine("sqlite:///events.db")
#create session to cache commands for sqlite engine instance
Session = sessionmaker(bind = engine)
session = Session()

#provide table definition
Base = declarative_base()

class Event(Base):
    __tablename__ = 'event'
    id = Column('id',Integer, primary_key = True)
    district = Column(Integer)
    title = Column(String(100))
    date = Column(String(50))
    details = Column(String(1000))
    time = Column(String(50))

    def __init__(self, title, date, details,time,district):
        self.title = title
        self.date = date
        self.details = details
        self.time = time
        self.district = district
        
    #for print    
    def __repr__(self):
        return f'{self.title} - {self.date}: {self.time}\n {self.details}'

In [17]:
page = requests.get('https://www.cb5.org/cb5m/calendar/2020-may/')
soup = BeautifulSoup(page.text,'html.parser')

In [18]:
req = urlopen("https://www.cb5.org/cb5m/calendar.ics").read()
cal = Calendar.from_ical(req)
events_dict = {}

for i, event in enumerate(cal.walk('vevent')):
    date = event.get('dtstart')
    time=  date.dt.hour
    if(date.dt.hour> 12):
        time = time - 12 
        time = str(time)+ ":"+str(date.dt.minute).zfill(2)+" PM"
    location = event.get('location')
    topic = event.get('summary')
    description = event.get('DESCRIPTION')
    description_id = description.split('/')[-2] 
    div = soup.findAll("div", {"id":description_id })
    
    for d in div:
        details = d.findAll('ul')      
    details_text =  '\n'.join([item.text for item in details if item.text.strip()])        
    
    events_dict[i] = {
        'date': str(date.dt.month) + "/"+ str(date.dt.day),
        'time': time,
        'topic': str(topic).replace('CB5 -','').strip(), #remove 'CB5 -' text
        'location': str(location),
        'details':details_text
    }

In [19]:
district = 105
#remove previous entries
session.query(Event).filter(Event.district == district).delete()
session.commit()

#add items to database
for event in events_dict.values():
    row = Event(title=event['topic'], date=event['date'],
                details= f'Location: {event["location"]}\n{event["details"]}',
                time=event['time'], district= district)
    session.add(row)
session.commit()

In [20]:
#print all users
for event in session.query(Event).filter(Event.district == 105):
    print(event)

Executive - 5/12: 5:00 PM
 Location: 

Full Board - 5/14: 6:00 PM
 Location: Videoconference
https://bit.ly/3eYmwoH

Budget, Education & City Services - 5/26: 6:00 PM
 Location: SGI-USA NY Culture Center
7 East 15th Street

Public Safety & Quality of Life - 5/27: 6:00 PM
 Location: NYC Seminar & Conference Center
71 West 23rd Street, 5th Floor

Transportation & Environment - 5/28: 6:00 PM
 Location: Bryant Park Corporation
111 West 40th Street, #2400

Parks & Public Spaces - 6/1: 6:00 PM
 Location: Bryant Park Corporation
111 West 40th Street, #2400

[Parks & Public Spaces Committee ONLY] Presentation by the Parks Department on an upcoming temporary public art installation in Union Square for June 2020.

[Joint Parks and Landmarks] Presentation by the Central Park Conservancy on four, in-kind capital projects:


Conservatory Garden - "Located along 5th Avenue between E. 104th-106th Street this is an in-kind restoration of the garden's infrastructure, paving, fence and masonry. Limited 